In [1]:
import os
from tqdm import tqdm

import cv2
import numpy as np
import matplotlib.pyplot as plt

from utils.dataloader import DataLoader
from utils.vis import MatplotlibVisualizer
from utils.transforms import HairRemoval, Composer
from utils.utils import export_experiment
from descriptors.shape import HOGDescriptor
from utils.segmentation import ThresholdingSegmentation
from descriptors.stats import IntensityStatsGridDescriptor
from descriptors.texture import LBPDescriptor, GLCMDescriptor, GaborFilterDescriptor
from descriptors.color import ColorDescriptor, ColorLayoutDescriptor, ColorCooccurrenceMatrixDescriptor

In [2]:
## Classes
CLASSES = ['nevus', 'others']

## Work folfer
work_folder = os.getcwd()
data_folder = os.path.join(work_folder, '..', 'Data/Challenge1')

## Visualizer
matplotlib_visualizer = MatplotlibVisualizer()
exp_name = 'binary_classification'

In [3]:
transforms_composer = Composer([
                                HairRemoval(),
                                ])

## Featrure Extraction

### Descriptors

In [4]:
## Define parameters
params = {
    'color_layout': {
        'grid_x': 3,
        'grid_y': 3,
    },
    'intensity_stats': {
        'grid_x': 3,
        'grid_y': 3,
    },
    'color': {
        'bins': (8, 12, 3),
        'grid_x': 3,
        'grid_y': 3,
    },
    'glcm': {
        'distances': [1],
        'angles': [0, np.pi/4, np.pi/2, 3*np.pi/4],
        'levels': 8,
        'grid_x': 3,
        'grid_y': 3,
    },
    'lbp': {
        'radius': 3,
        'n_points': 16,
        'grid_x': 3,
        'grid_y': 3,
    },
}

In [ ]:
modes = ['train', 'val']

## Descriptors
color_layout_descriptor = ColorLayoutDescriptor(**params['color_layout'])
intensity_stats_grid_descriptor = IntensityStatsGridDescriptor(**params['intensity_stats'])
color_descriptor = ColorDescriptor(**params['color'])
glcm_descriptor = GLCMDescriptor(**params['glcm'])
lbp_descriptor = LBPDescriptor(**params['lbp'])

features_dict = {}

for mode in modes:

    ## Data loader
    dataloader = DataLoader(data_folder, mode, 
                            shuffle=True, 
                            ignore_folders=['black_background', '.DS_Store'], 
                            max_samples=None, 
                            balance=False,
                            transforms=None, 
                            classes=CLASSES, 
                            mask=False)

    ## Extract features
    features = []
    labels = []
    for i, (img, label, mask, path) in tqdm(enumerate(dataloader), total=len(dataloader), desc=f'Extracting features for {mode}'):
        color_features = color_descriptor.extract(img, mask=None)
        intensity_stats_grid_features = intensity_stats_grid_descriptor.extract(img, mask=None)
        glcm_features, glcm_img = glcm_descriptor.extract(img, mask=None)
        lbp_features, lbp_img = lbp_descriptor.extract(img, mask=None)
        features.append(np.concatenate([intensity_stats_grid_features, color_features, lbp_features, glcm_features], axis=0))        
        
        ## add label
        labels.append(label)
        
    
    ## Save features to disk
    features = np.array(features)
    labels = np.array(labels)
    features_with_labels = np.concatenate([features, labels.reshape(-1, 1)], axis=1)
    features_dict[mode] = features_with_labels

Extracting features for val: 100%|██████████| 3796/3796 [06:40<00:00,  9.48it/s]


## Training

In [28]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import xgboost as xgb

In [25]:
mode = 'train'
# model = SVC(kernel='rbf', C=5.0, random_state=42, degree=5)
xgb_clf = xgb.XGBClassifier(objective='binary:logistic', n_estimators=1000, learning_rate=0.2, n_jobs=-1)

In [26]:
features = features_dict[mode][:, :-1]
labels = features_dict[mode][:, -1]

X_train, X_val, y_train, y_val = train_test_split(features, labels, test_size=0.2, random_state=42, stratify=labels)

In [ ]:
scaler = MinMaxScaler()
model = Pipeline([('scaler', scaler), ('model', xgb_clf)])

In [34]:
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.2, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, n_jobs=-1,
              num_parallel_tree=None, random_state=None, ...)

## Validation

In [35]:
y_pred = model.predict(X_val)
print(classification_report(y_val, y_pred, target_names=CLASSES))

              precision    recall  f1-score   support

       nevus       0.85      0.85      0.85      1545
      others       0.84      0.85      0.85      1494

    accuracy                           0.85      3039
   macro avg       0.85      0.85      0.85      3039
weighted avg       0.85      0.85      0.85      3039



## Testing

In [36]:
mode = 'val'
features_test = features_dict[mode][:, :-1]
labels_test = features_dict[mode][:, -1]

y_pred = model.predict(features_test)
print(classification_report(labels_test, y_pred, target_names=CLASSES))

              precision    recall  f1-score   support

       nevus       0.84      0.84      0.84      1931
      others       0.83      0.84      0.84      1865

    accuracy                           0.84      3796
   macro avg       0.84      0.84      0.84      3796
weighted avg       0.84      0.84      0.84      3796



## Cross Validation

In [14]:
from sklearn.model_selection import cross_val_score, cross_val_predict, StratifiedKFold

# Initialize XGBoost classifier
xgb_clf = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', objective='binary:logistic', n_estimators=500, learning_rate=0.1, n_jobs=-1)

# Define cross-validation strategy (e.g., 5-fold stratified)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation and get accuracy scores for each fold
features_train = features_dict['train'][:, :-1]
labels_train = features_dict['train'][:, -1]
cv_scores = cross_val_score(xgb_clf, features_train, labels_train, cv=cv, scoring='accuracy')

# Output the results
print("Cross-Validation Accuracy Scores:", cv_scores)
print("Mean CV Accuracy:", cv_scores.mean())
print("Standard Deviation of CV Accuracy:", cv_scores.std())

Cross-Validation Accuracy Scores: [0.5 0.4 0.5 0.4 0.5]
Mean CV Accuracy: 0.45999999999999996
Standard Deviation of CV Accuracy: 0.04898979485566355


In [15]:
# Cross-validation predictions (optional)
features_test = features_dict['val'][:, :-1]
labels_test = features_dict['val'][:, -1]
cv_predictions = cross_val_predict(xgb_clf, features_test, labels_test, cv=cv)
print("Classification Report for CV Predictions:\n", classification_report(labels_test, cv_predictions))

Classification Report for CV Predictions:
               precision    recall  f1-score   support

         0.0       0.70      0.76      0.73        25
         1.0       0.74      0.68      0.71        25

    accuracy                           0.72        50
   macro avg       0.72      0.72      0.72        50
weighted avg       0.72      0.72      0.72        50



## Grid Search

In [37]:
from sklearn.model_selection import GridSearchCV

# Initialize the base model (XGBClassifier)
xgb_clf = xgb.XGBClassifier(objective='binary:logistic', n_jobs=-1, random_state=42)
scaler_cv = StandardScaler()

model = Pipeline([('scaler', scaler_cv), ('model', xgb_clf)])

# Define the parameter grid for Grid Search
param_grid = {
    'model__n_estimators': [1000, 1500],  # Number of trees
    'model__learning_rate': [0.1, 0.2],  # Step size shrinkage
    # 'reg_lambda': [1.0, 0.8],  # L2 regularization term on weights
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, 
                        scoring='accuracy',  # Use 'accuracy' as the evaluation metric
                        cv=5,  # 5-fold cross-validation
                        verbose=4, 
                        n_jobs=-1)  # Parallel processing

features_train = features_dict['train'][:, :-1]
labels_train = features_dict['train'][:, -1]
grid_search.fit(features_train, labels_train)

# Get the best parameters and best score from the grid search
print("Best Parameters: ", grid_search.best_params_)
print("Best Accuracy: ", grid_search.best_score_)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 4/5] END model__learning_rate=0.1, model__n_estimators=1000;, score=0.833 total time= 7.0min
[CV 3/5] END model__learning_rate=0.1, model__n_estimators=1000;, score=0.844 total time= 7.1min
[CV 1/5] END model__learning_rate=0.1, model__n_estimators=1000;, score=0.843 total time= 7.2min
[CV 5/5] END model__learning_rate=0.1, model__n_estimators=1000;, score=0.847 total time= 7.2min
[CV 2/5] END model__learning_rate=0.1, model__n_estimators=1000;, score=0.845 total time= 7.3min
[CV 3/5] END model__learning_rate=0.1, model__n_estimators=1500;, score=0.846 total time= 9.9min
[CV 1/5] END model__learning_rate=0.1, model__n_estimators=1500;, score=0.845 total time=10.0min
[CV 2/5] END model__learning_rate=0.1, model__n_estimators=1500;, score=0.848 total time=10.1min
[CV 1/5] END model__learning_rate=0.2, model__n_estimators=1000;, score=0.849 total time= 6.1min
[CV 2/5] END model__learning_rate=0.2, model__n_estimators=1000;, s

In [38]:
best_model = grid_search.best_estimator_
features_test = features_dict['val'][:, :-1]
labels_test = features_dict['val'][:, -1]

y_pred = best_model.predict(features_test)
print(classification_report(labels_test, y_pred, target_names=CLASSES))

              precision    recall  f1-score   support

       nevus       0.85      0.85      0.85      1931
      others       0.85      0.85      0.85      1865

    accuracy                           0.85      3796
   macro avg       0.85      0.85      0.85      3796
weighted avg       0.85      0.85      0.85      3796



## Exporting Experiment

In [40]:
## Export experiment
notebook_name = 'BinaryClass.ipynb'
export_experiment(name=exp_name, params=params, feature_dict=features_dict , model=best_model, notebook_name=notebook_name)

<IPython.core.display.Javascript object>

Experiment 'binary_classification' saved at experiments/binary_classification_20241029_165927


## Feature Selection

In [20]:
features_train = features_dict['train'][:, :-1]
labels_train = features_dict['train'][:, -1]
X_train, X_val, y_train, y_val = train_test_split(features_train, labels_train, test_size=0.1, random_state=42, stratify=labels)

In [21]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd


# Train random forest and get feature importances
model = RandomForestClassifier()
model.fit(X_train, y_train)
importances = model.feature_importances_

# Display feature importances
feature_importances = pd.Series(importances)
print(feature_importances.sort_values(ascending=False))

2786    0.010357
1379    0.010292
1457    0.009750
1454    0.009593
590     0.009396
          ...   
1019    0.000000
1020    0.000000
1021    0.000000
1022    0.000000
2834    0.000000
Length: 2835, dtype: float64


In [22]:
#get the most important features
top_features = feature_importances.sort_values(ascending=False).index[:400]

# Retrain the model using only the top features
model = xgb.XGBClassifier(objective='binary:logistic', n_estimators=1000, learning_rate=0.2, n_jobs=-1)
model.fit(X_train[:, top_features], y_train)

# Evaluate the model
y_pred = model.predict(X_val[:, top_features])
print(classification_report(y_val, y_pred, target_names=CLASSES))

              precision    recall  f1-score   support

       nevus       1.00      0.67      0.80         3
      others       0.67      1.00      0.80         2

    accuracy                           0.80         5
   macro avg       0.83      0.83      0.80         5
weighted avg       0.87      0.80      0.80         5



In [23]:
# test the model
features_test = features_dict['val'][:, :-1]
labels_test = features_dict['val'][:, -1]

y_pred = model.predict(features_test[:, top_features])
print(classification_report(labels_test, y_pred, target_names=CLASSES))

              precision    recall  f1-score   support

       nevus       0.58      0.84      0.69        25
      others       0.71      0.40      0.51        25

    accuracy                           0.62        50
   macro avg       0.65      0.62      0.60        50
weighted avg       0.65      0.62      0.60        50

